REST (Representational State Transfer) is an architectural style for designing networked applications. In a
RESTful API, resources are represented as URIs (Uniform Resource Identifiers), and different HTTP methods
are used to perform operations on these resources:

• GET: Retrieves one or more resources. Used for reading data.

• POST: Creates a new resource. Used for creating data.

• PUT: Updates an existing resource. Used for modifying data.

• DELETE: Deletes a resource. Used for removing data.


HTTP status codes are used to indicate the outcome of each request. Some common status codes include:

• 200 OK: The request was successful.

• 201 Created: The resource was successfully created.

• 400 Bad Request: The request was malformed or invalid.

• 404 Not Found: The requested resource was not found.

• 500 Internal Server Error: An unexpected error occurred on the server.

Now let’s learn how to query the REST Countries API using Python. This API provides information about countries, including their names, capitals, populations, and more. To retrieve information about a specific country (Belgium) by its name, we can make a GET request to the
following endpoint

In [6]:
from requests import put, get
response = get('https://restcountries.com/v3.1/name/belgium')
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    belgium = response.json()
    print(belgium)
else:
    print("Failed to retrieve country information. Status code:", response.status_code)

[{'name': {'common': 'Belgium', 'official': 'Kingdom of Belgium', 'nativeName': {'deu': {'official': 'Königreich Belgien', 'common': 'Belgien'}, 'fra': {'official': 'Royaume de Belgique', 'common': 'Belgique'}, 'nld': {'official': 'Koninkrijk België', 'common': 'België'}}}, 'tld': ['.be'], 'cca2': 'BE', 'ccn3': '056', 'cca3': 'BEL', 'cioc': 'BEL', 'independent': True, 'status': 'officially-assigned', 'unMember': True, 'currencies': {'EUR': {'name': 'Euro', 'symbol': '€'}}, 'idd': {'root': '+3', 'suffixes': ['2']}, 'capital': ['Brussels'], 'altSpellings': ['BE', 'België', 'Belgie', 'Belgien', 'Belgique', 'Kingdom of Belgium', 'Koninkrijk België', 'Royaume de Belgique', 'Königreich Belgien'], 'region': 'Europe', 'subregion': 'Western Europe', 'languages': {'deu': 'German', 'fra': 'French', 'nld': 'Dutch'}, 'translations': {'ara': {'official': 'مملكة بلجيكا', 'common': 'بلجيكا'}, 'bre': {'official': 'Rouantelezh Belgia', 'common': 'Belgia'}, 'ces': {'official': 'Belgické království', 'com

This API is accessible via multiple endpoints. This means that we can perform different queries. For instance,we could query for all french speaking countries:

In [7]:
french_speaking_countries = get('https://restcountries.com/v3.1/lang/french')
french_speaking_countries = french_speaking_countries.json()

## Building a restful API with Flask

Example

In [8]:
from flask import Flask
from flask_restful import Resource, Api
app = Flask(__name__)
api = Api(app)
class HelloWorld(Resource):
    def get(self):
        return {'hello': 'world'}
api.add_resource(HelloWorld, '/')
if __name__ == '__main__':
    app.run(debug=True) 

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/home/antonio/Desktop/Geospatial/lab/labgeospatial/.venv/lib/python3.8/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/antonio/Desktop/Geospatial/lab/labgeospatial/.venv/lib/python3.8/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/antonio/Desktop/Geospatial/lab/labgeospatial/.venv/lib/python3.8/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/antonio/Desktop/Geospatial/lab/labgeospatial/.venv/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/antonio/Desktop/Geospatial/lab/labgeospatial/.venv/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socke

SystemExit: 1

## Accessing data

Lets adapt our HelloWorld app by initialize a contact book dictionary containing a single entry and create a
simple ressource for accessing it:

In [ ]:
contacts = {"gilles": {"email":"gilles.dejaegere@ulb.be", "office": "UB4.131"}}
class Contact(Resource):
    def get(self, contact_name):
        return contacts[contact_name]
api.add_resource(Contact, '/contact/<string:contact_name>')

Since we launched the app in debug mode, the server should take into account the modification in your file once
it is saved. If not, re-launch your script. You can then test via another python script (such as in the fist section
of this lab):

In [ ]:
response = get('https://127.0.0.1:5000/contact/gilles')

## Adding data and Parsing input

In [ ]:
from flask_restful import Api, Resource, reqparse, abort
contacts = {"gilles": {"email":"gilles.dejaegere@ulb.be", "office": "UB4.131"}}
contact_args = reqparse.RequestParser()
contact_args.add_argument("email", type=str, help="Email required", required=True)
contact_args.add_argument("office", type=str, help="Office is required", required=True)

In this code we specify that the two fields are strings, that they are required and also provide some help message
for in case the user makes an error.
This request parser can then be used in the different methods of our Contact ressource. Let’s use it to create a
"put" method:

In [ ]:
class Contact(Resource):
    def get(self, contact_name):
        return contacts[contact_name]
    def put(self, contact_name):
        if contact_name in contacts:
            abort(409, message=f"{contact_name} already in contacts")
        args = contact_args.parse_args()
        contact = {"email":args['email'], "office":args['office']}
        contacts[contact_name] = contact

Two elements can be noticed. First some code has been added to abort the request if the user try to add
a already existing contact. Secondly, the contact_args object is used to parse the data provided with the
request.
We can now test this code. For this, let’s try to add some new contacts via the following script:

In [ ]:
import requests
headers = {
    'Content-type':'application/json',
    'Accept':'application/json'
}
response = requests.put("http://127.0.0.1:5000/contact/jean",
                        json={"email": "jean@ulb.be", "office":"UB4.132"},
                        headers=headers)
print(response.json())

## 3.3 Multiple Endpoints

In [ ]:
class Contacts(Resource):
    def get(self):
        return contacts
api.add_resource(Contacts, '/contacts')

This last functionality can be tested by accessing the new endpoint:

In [ ]:
response = requests.get("http://127.0.0.1:5000/contacts")
print(response.json())

## Exercice

Complete the notebook API. For instance, you should add methods for:

• Deleting a contact;

In [ ]:
def delete(self, contact_name):
    if contact_name not in contacts:
        abort(404, message=f"{contact_name} not in contacts")
    del contacts[contact_name]
    return '', 204

• Modifying a contact;

In [ ]:
def modify(self, contact_name):
    if contact_name not in contacts:
        abort(404, message=f"{contact_name} not in contacts")
    args = contact_args.parse_args()
    contacts[contact_name] = {"email":args['email'], "office":args['office']}
    return contacts[contact_name], 201

• Save the contact book in a csv file;

In [ ]:
def save(self):
    with open('contacts.csv', 'w') as f:
        for contact in contacts:
            f.write(f"{contact},{contacts[contact]['email']},{contacts[contact]['office']}\n")

## Additional resources
Hereunder are listed a few resources about Flask (from which this session was inspired).
Written resources:


• https://flask.palletsprojects.com/en/3.0.x/quickstart/ : Quickstart about Flask to build webpages.

• https://flask-restful.readthedocs.io/en/latest/quickstart.html : How to build a restful API with flask.

• https://www.youtube.com/watch?v=GMppyAPbLYk : Good video covering most of the basic needs with a concrete example